## Lab6-Assignment: Topic Classification

Use the same training, development, and test partitions of the the 20 newsgroups text dataset as in Lab6.4-Topic-classification-BERT.ipynb

* Fine-tune and examine the performance of another transformer-based pretrained language models, e.g., RoBERTa, XLNet

* Compare the performance of this model to the results achieved in Lab6.4-Topic-classification-BERT.ipynb and to a conventional machine learning approach (e.g., SVM, Naive Bayes) using bag-of-words or other engineered features of your choice.
Describe the differences in performance in terms of Precision, Recall, and F1-score evaluation metrics.

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00


In [3]:
!pip install wandb

In [4]:
# imports
import torch
import pandas as pd
import numpy as np
import transformers
import wandb
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from datasets import Dataset

In [5]:
###copy paste from lab 6.4

# load only a sub-selection of the categories (4 in our case)
categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'sci.space']

# remove the headers, footers and quotes (to avoid overfitting)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories, random_state=42)

In [6]:
###copy paste from lab 6.4
train = pd.DataFrame({'text': newsgroups_train.data, 'labels': newsgroups_train.target})
test = pd.DataFrame({'text': newsgroups_test.data, 'labels': newsgroups_test.target})

In [7]:
###copy paste from lab 6.4
train, dev = train_test_split(train, test_size=0.1, random_state=0, stratify=train[['labels']])

In [8]:
# pandas df to hugging face datasetformat
train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)
test_dataset = Dataset.from_pandas(test)

In [9]:
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, using CPU.")

Using GPU: Tesla T4


In [10]:
# XLNet
model_name = "xlnet-base-cased"
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

#tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["text"])
dev_dataset = dev_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

train_dataset.set_format("torch")
dev_dataset.set_format("torch")
test_dataset.set_format("torch")


Map:   0%|          | 0/2025 [00:00<?, ? examples/s]

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/1498 [00:00<?, ? examples/s]

In [12]:
# compute metrics that will be used after training
def compute_metrics(pred):
    predictions = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(pred.label_ids, predictions, average='weighted')
    accuracy = accuracy_score(pred.label_ids, predictions)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [15]:
training_args = TrainingArguments(
    output_dir="./xlnet_results",
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    fp16=True,
    num_train_epochs=10,  #training epochs: maybe increase
    weight_decay=0.01,
    logging_dir="./logs_xlnet",
    logging_steps=10,  #log every 10 steps
    save_total_limit=2,
    load_best_model_at_end=True
)


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

<ipython-input-16-c51e17a7962a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,1.533500,0.350211,0.889381,0.893865,0.889381,0.888687
1,1.001100,0.455818,0.845133,0.871593,0.845133,0.846777
2,0.693000,0.451654,0.889381,0.900604,0.889381,0.890558
3,0.476000,0.444651,0.915929,0.917516,0.915929,0.915619
4,0.275300,0.488163,0.907080,0.912222,0.907080,0.907421
5,0.340100,0.408598,0.929204,0.932964,0.929204,0.929470
6,0.160500,0.393538,0.924779,0.925675,0.924779,0.924049
7,0.036700,0.528435,0.915929,0.916240,0.915929,0.915430
8,0.064100,0.538252,0.911504,0.914599,0.911504,0.911664
9,0.134000,0.511188,0.915929,0.918932,0.915929,0.916045


TrainOutput(global_step=1260, training_loss=0.5061547558459024, metrics={'train_runtime': 2771.7328, 'train_samples_per_second': 7.306, 'train_steps_per_second': 0.455, 'total_flos': 5753537546600448.0, 'train_loss': 0.5061547558459024, 'epoch': 9.994082840236686})